## 1. Bert Classification (신문 주제 분류기)

In [57]:
# ============================================================================
# 1. 필수 패키지 설치
# ============================================================================
print("="*80)
print("🔧 1. 필수 패키지 설치")
print("="*80)
!pip install -q --upgrade transformers datasets accelerate scikit-learn polars

# 한글 깨짐 현상 해결
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf
print("✅ 패키지 설치 완료\n")


# ============================================================================
# 2. 전체 라이브러리 임포트
# ============================================================================
print("="*80)
print("📚 2. 전체 라이브러리 임포트")
print("="*80)
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"
import sys
import torch
import numpy as np
import pandas as pd
import polars as pl
from pathlib import Path
import pickle
import gc
import shutil
import time
from google.colab import drive
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm

# Scikit-learn 모듈
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix

# Hugging Face 핵심 모듈
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    EarlyStoppingCallback
)

print(f"✅ Torch version: {torch.__version__}")
print(f"✅ CUDA available: {torch.cuda.is_available()}")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"✅ 사용 디바이스: {device}\n")

# (★ ★ ★ A100 최적화 ★ ★ ★)
# A100의 고성능 CPU 코어 수에 맞춰 map/dataloader에 사용할 프로세스 수 정의
N_PROC = 16


# ============================================================================
# 3. Google Drive 마운트 및 데이터 로컬 복사 (★ ★ ★ RAM 디스크 최적화 ★ ★ ★)
# ============================================================================
print("="*80)
print("📁 3. Google Drive 마운트 및 데이터 로컬 복사 (RAM 디스크)")
print("="*80)
drive.mount('/content/drive', force_remount=False)

SAVE_BASE_PATH = Path("/content/drive/MyDrive/best_news_classifier")
SAVE_BASE_PATH.mkdir(parents=True, exist_ok=True)
print(f"✅ 저장 경로 확인: {SAVE_BASE_PATH}")

# 원본 데이터 경로 탐색
DRIVE_BASE_PATH = Path("/content/drive/MyDrive/joongang_crawl")
DRIVE_FILE_PATH = DRIVE_BASE_PATH / "combined_articles.parquet"
if not DRIVE_FILE_PATH.exists():
    DRIVE_FILE_PATH = Path("/content/drive/MyDrive/combined_articles.parquet")
if not DRIVE_FILE_PATH.exists():
    raise FileNotFoundError("❌ 구글 드라이브에서 combined_articles.parquet 파일을 찾을 수 없습니다.")

# (★ ★ ★ 핵심 수정 ★ ★ ★)
# 느린 /content/ 디스크 대신, 초고속 /dev/shm/ (RAM 디스크)에 복사합니다.
# A100의 I/O 병목 현상("뭘 해도 1시간 46분")을 해결하기 위한 조치입니다.
LOCAL_FILE_PATH = Path("/dev/shm/combined_articles.parquet")

if not LOCAL_FILE_PATH.exists():
    print(f"🚀 데이터 복사 시작: Drive -> RAM Disk (/dev/shm/)...")
    start_time = time.time()
    shutil.copyfile(DRIVE_FILE_PATH, LOCAL_FILE_PATH)
    print(f"✅ 데이터 복사 완료 ({time.time() - start_time:.1f}초 소요)")
else:
    print("✅ (스킵) RAM 디스크에 이미 데이터가 존재합니다. 복사를 건너뜁니다.")

# RAM 디스크에서 데이터를 로드하므로 디스크 I/O가 발생하지 않습니다.
raw_dataset = load_dataset("parquet", data_files={"train": str(LOCAL_FILE_PATH)}, split="train")
print(f"✅ Hugging Face Dataset 로드 완료 (RAM Mapped): {len(raw_dataset):,} 건")

# RAM 디스크 사용 현황 확인
!df -h /dev/shm/


# ============================================================================
# 4. 스마트한 시계열(Time-Series) 데이터 분할
# ============================================================================
print("\n" + "="*80)
print("📅 4. 스마트 시계열 데이터 분할 (RAM 최적화)")
print("="*80)

print("... 날짜 기반 인덱싱 계산 중 ...")
# 날짜와 라벨만 빠르게 로드
meta_df = raw_dataset.select_columns(['yyyymmdd', 'sector1']).to_pandas()

valid_indices = meta_df[
    (meta_df['yyyymmdd'].notna()) &
    (meta_df['sector1'].notna()) &
    (meta_df['yyyymmdd'].astype(float) >= 20200101) &
    (meta_df['yyyymmdd'].astype(float) <= 20251231)
].index.to_numpy()

sorted_indices = valid_indices[np.argsort(meta_df.loc[valid_indices, 'yyyymmdd'].values)]

total_len = len(sorted_indices)
train_end = int(total_len * 0.8)
val_end = int(total_len * 0.9)

train_idx = sorted_indices[:train_end]
val_idx = sorted_indices[train_end:val_end]
test_idx = sorted_indices[val_end:]

print(f"✅ 전체 유효 데이터: {total_len:,} 건")
print(f" - Train: {len(train_idx):,} 건 (80%)")
print(f" - Valid: {len(val_idx):,} 건 (10%)")
print(f" - Test : {len(test_idx):,} 건 (10%)")

del meta_df, valid_indices, sorted_indices
gc.collect()


# ============================================================================
# 5. [핵심 개선] 라벨 집계(Aggregation) 및 인코딩 (V2)
# ============================================================================
print("\n" + "="*80)
print("🏷️ 5. [핵심 개선] 라벨 집계(Aggregation) 및 인코딩 (V2)")
print("="*80)

def map_to_main_sector(sector_str):
    if not isinstance(sector_str, str):
        return '기타' # None, NaN 등 처리
    if sector_str in ['경제', '부동산', '돈 버는 재미', '머니랩']:
        return '경제'
    if sector_str in ['정치', '더 북한']:
        return '정치'
    if sector_str in ['사회', '피플', '세상과 함께', '가족과 함께', 'hello! Parents', '톡톡에듀']:
        return '사회'
    if sector_str in ['국제', '더 차이나']:
        return '국제'
    if sector_str in ['스포츠', '문화', '여행레저', 'COOKING', '쉴 땐 뭐하지',
                      '더,마음', '2024 파리올림픽', '더,오래', '더 헬스', '라이프',
                      '마음 챙기기', '더 하이엔드', '비크닉']:
        return '문화/스포츠'
    if sector_str in ['브랜드뉴스', '오피니언', '중앙SUNDAY', 'Leader & Reader', 'Tran:D']:
        return '기타'
    return '기타'

# 5-1. Train Set의 원본 라벨로 집계 및 인코더 학습
print("... 원본 라벨을 6개 메인 섹터로 집계 중 ...")
train_labels_raw = raw_dataset.select(train_idx)['sector1']
agg_labels_raw = [map_to_main_sector(s) for s in train_labels_raw]

label_encoder = LabelEncoder()
label_encoder.fit(agg_labels_raw) # 집계된 라벨로 'fit'
num_labels = len(label_encoder.classes_)
print(f"✅ [개선] 33개 라벨 -> {num_labels}개 메인 라벨로 집계 완료")
print(f"   (클래스: {label_encoder.classes_})")

# 5-2. 안정적인 영문 라벨 맵 생성
KOR_TO_ENG_MAP = {
    '경제': 'Economy',
    '국제': 'International',
    '기타': 'ETC',
    '문화/스포츠': 'Culture/Sports',
    '사회': 'Society',
    '정치': 'Politics'
}
english_labels_ordered = [KOR_TO_ENG_MAP.get(k, k) for k in label_encoder.classes_]
print(f"   (영문 라벨: {english_labels_ordered})")


# 5-3. 집계된 라벨 분포 (불균형 확인)
print("\n📊 [중요] 집계된 라벨 분포 (Train Set):")
agg_dist_series = pd.Series(agg_labels_raw)
print(agg_dist_series.value_counts(normalize=True).to_string(float_format="{:.2%}".format))

del train_labels_raw, agg_labels_raw, agg_dist_series
gc.collect()

# 5-4. 최종 데이터 전처리 함수 (집계 + 인코딩 포함)
def preprocess_and_encode(examples):
    h = [t if t else "" for t in examples['headline']]
    c = [t if t else "" for t in examples['content']]
    texts = [f"{hh} {cc}"[:1000] for hh, cc in zip(h, c)] # 512 토큰에 맞춰 적절히 자름
    main_sectors = [map_to_main_sector(s) for s in examples['sector1']]
    labels = label_encoder.transform(main_sectors)
    return {'text': texts, 'label': labels}

# 5-5. 데이터셋 생성 및 전처리 적용
train_ds = raw_dataset.select(train_idx)
val_ds = raw_dataset.select(val_idx)
test_ds = raw_dataset.select(test_idx)

print(f"\n... 데이터셋 전처리 및 라벨 집계 적용 중 (N_PROC={N_PROC})...")
train_ds_processed = train_ds.map(preprocess_and_encode, batched=True, num_proc=N_PROC, remove_columns=raw_dataset.column_names)
val_ds_processed = val_ds.map(preprocess_and_encode, batched=True, num_proc=N_PROC, remove_columns=raw_dataset.column_names)
test_ds_processed = test_ds.map(preprocess_and_encode, batched=True, num_proc=N_PROC, remove_columns=raw_dataset.column_names)
print("✅ 데이터셋 전처리 완료 (텍스트 + 6개 집계 라벨)\n")


# ============================================================================
# 6. [베이스라인] TF-IDF + 로지스틱 회귀 (★ 스킵 기능 적용)
# ============================================================================
print("="*80)
print("🔬 6. [베이스라인] TF-IDF + 로지스틱 회귀 (스킵 기능 적용)")
print("="*80)

LR_MODEL_PATH = SAVE_BASE_PATH / "baseline_lr.pkl"
LR_VEC_PATH = SAVE_BASE_PATH / "baseline_tfidf_vec.pkl"
LR_REPORT_PATH = SAVE_BASE_PATH / "baseline_report_lr.pkl"

X_test_text = test_ds_processed['text']
y_test_baseline = test_ds_processed['label']

if LR_MODEL_PATH.exists() and LR_VEC_PATH.exists() and LR_REPORT_PATH.exists():
    print("✅ (스킵) [6-2, 6-3] 훈련된 TF-IDF 및 LR 모델 로드...")
    with open(LR_VEC_PATH, 'rb') as f:
        tfidf_vectorizer = pickle.load(f)
    with open(LR_MODEL_PATH, 'rb') as f:
        lr_clf = pickle.load(f)
    with open(LR_REPORT_PATH, 'rb') as f:
        report_lr = pickle.load(f)

    print("... Test Set에 TF-IDF 적용 중 ...")
    X_test_tfidf = tfidf_vectorizer.transform(X_test_text)
    print("✅ 로드된 벡터라이저로 Test Set 변환 완료")

else:
    print("... [6-1] 데이터셋에서 텍스트/라벨 추출 중 (최초 1회 실행) ...")
    X_train_text = train_ds_processed['text']
    y_train = train_ds_processed['label']
    print(f"✅ Train: {len(X_train_text):,} 건, Test: {len(X_test_text):,} 건 로드 완료")

    # 6-2. TF-IDF 벡터화
    print("\n🚀 [6-2] TF-IDF 벡터화 중 (Train Set 기준)...")
    tfidf_vectorizer = TfidfVectorizer(min_df=5, max_features=50000, sublinear_tf=True)
    X_train_tfidf = tfidf_vectorizer.fit_transform(X_train_text)
    X_test_tfidf = tfidf_vectorizer.transform(X_test_text)
    print(f"✅ TF-IDF 벡터화 완료. 피처 개수: {X_train_tfidf.shape[1]} 개")

    # 6-3. 로지스틱 회귀 학습 (불균형 처리)
    print("\n🚀 [6-3] 로지스틱 회귀 분류기 학습 중...")
    lr_clf = LogisticRegression(
        solver='saga',
        C=1.0,
        max_iter=1000,
        n_jobs=-1,
        random_state=42,
        class_weight='balanced'
    )
    lr_clf.fit(X_train_tfidf, y_train)
    print("✅ 로지스틱 회귀 학습 완료 (불균형 가중치 적용)")

    print(f"... 베이스라인 모델 저장 중: {SAVE_BASE_PATH}")
    with open(LR_VEC_PATH, 'wb') as f:
        pickle.dump(tfidf_vectorizer, f)
    with open(LR_MODEL_PATH, 'wb') as f:
        pickle.dump(lr_clf, f)

    del X_train_text, y_train, X_train_tfidf
    gc.collect()

# 6-4. 평가 및 비교
print("\n📊 [6-4] [TF-IDF + 로지스틱 회귀] Test Set 평가 결과:")
y_pred_baseline = lr_clf.predict(X_test_tfidf)
all_labels_indices = np.arange(len(label_encoder.classes_))

if 'report_lr' not in locals():
    print("... [최초 실행] Classification Report 생성 및 저장 ...")
    report_lr = classification_report(
        y_test_baseline,
        y_pred_baseline,
        labels=all_labels_indices,
        target_names=label_encoder.classes_,
        digits=4,
        zero_division=0,
        output_dict=True
    )
    with open(LR_REPORT_PATH, 'wb') as f:
        pickle.dump(report_lr, f)

print(classification_report(
    y_test_baseline,
    y_pred_baseline,
    labels=all_labels_indices,
    target_names=label_encoder.classes_,
    digits=4,
    zero_division=0
))
print("\n✅ 베이스라인(LR) 모델 평가 결과가 'report_lr' 변수에 저장되었습니다.")

del X_test_text, y_test_baseline, X_test_tfidf, y_pred_baseline
gc.collect()


# ============================================================================
# 7. [딥러닝 모델] 모델 및 토크나이저 로드 (★ 버그 수정 완료 ★)
# ============================================================================
print("\n" + "="*80)
print("⚙️ 7. [딥러닝 모델] 모델 로드 (KLUE-RoBERTa)")
print("="*80)

MODEL_NAME = 'klue/roberta-base'
print(f"🤖 사용 모델: {MODEL_NAME}")

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=num_labels)
model.to(device)

# (★ ★ ★ 핵심 버그 수정 ★ ★ ★)
# klue/roberta-base의 최대 길이는 514 (512 + 2특수토큰) 입니다.
# 1024로 설정하면 'CUDA device-side assert' 오류가 발생합니다.
def tokenize_text(examples):
    return tokenizer(
        examples['text'],
        padding="max_length",
        truncation=True,
        max_length=512  # ★ 1024 -> 512로 수정
    )

print(f"... 토크나이징 진행 중 (max_length=512, N_PROC={N_PROC})...")
train_ds_tokenized = train_ds_processed.map(tokenize_text, batched=True, num_proc=N_PROC, remove_columns=['text'])
val_ds_tokenized = val_ds_processed.map(tokenize_text, batched=True, num_proc=N_PROC, remove_columns=['text'])
test_ds_tokenized = test_ds_processed.map(tokenize_text, batched=True, num_proc=N_PROC, remove_columns=['text'])

train_ds_tokenized.set_format("torch")
val_ds_tokenized.set_format("torch")
test_ds_tokenized.set_format("torch")
print("✅ 토크나이징 완료\n")


# ============================================================================
# 8. [딥러닝 모델] 파인튜닝 (★ ★ ★ 시스템 권장 최종 튜닝 ★ ★ ★)
# ============================================================================
print("="*80)
print("🔥 8. [딥러닝 모델] 파인튜닝 시작 (시스템 권장 튜닝)")
print("="*80)

# (VRAM 클리어 코드)
import gc
gc.collect()
torch.cuda.empty_cache()
print("✅ 이전 VRAM 캐시를 비웠습니다. 0GB에서 새로 시작합니다.")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    acc = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions, average='weighted')
    return {'accuracy': acc, 'f1': f1}

# [최적화된 학습 파라미터]
training_args = TrainingArguments(
    output_dir="/content/news-classifier-checkpoints",
    num_train_epochs=5,

    # (유지) VRAM을 75GB+로 채우는 배치
    per_device_train_batch_size=352,

    gradient_accumulation_steps=1,

    learning_rate=3e-5,
    weight_decay=0.01,

    eval_strategy="epoch",
    save_strategy="epoch",

    logging_steps=100,

    load_best_model_at_end=True,
    metric_for_best_model="f1",

    fp16=True,

    # (★ ★ ★ 최종 튜닝 ★ ★ ★)
    # 시스템이 16개는 과도하고 12개가 최적이라고 경고 (UserWarning)
    # 따라서 16 -> 12로 수정
    dataloader_num_workers=12,

    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds_tokenized,
    eval_dataset=val_ds_tokenized,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)

print(f"🚀 학습 시작 (Batch: {training_args.per_device_train_batch_size}, Workers: {training_args.dataloader_num_workers}, fp16: {training_args.fp16})...")
trainer.train()
print("✅ 학습 완료!\n")


# ============================================================================
# 9. [딥러닝 모델] 최종 평가
# ============================================================================
print("="*80)
print("🏆 9. [딥러닝 모델] 최종 모델 평가")
print("="*80)

print("... 딥러닝 모델 Test Set 평가 중 ...")
test_results = trainer.evaluate(test_ds_tokenized)
print(f"\n🎯 Test Set 정확도: {test_results['eval_accuracy']:.4f}")
print(f"🎯 Test Set F1점수: {test_results['eval_f1']:.4f}\n")

print("... 딥러닝 모델 Test Set 예측(Prediction) 생성 중 ...")
dl_predictions = trainer.predict(test_ds_tokenized)
y_pred_dl = np.argmax(dl_predictions.predictions, axis=1)
y_true_dl = dl_predictions.label_ids

# Classification Report 생성
report_dl = classification_report(
    y_true_dl,
    y_pred_dl,
    labels=all_labels_indices,
    target_names=label_encoder.classes_,
    digits=4,
    zero_division=0,
    output_dict=True
)
# 화면에도 리포트 출력
print("\n📊 [KLUE-RoBERTa] Test Set 평가 결과:")
print(classification_report(
    y_true_dl,
    y_pred_dl,
    labels=all_labels_indices,
    target_names=label_encoder.classes_,
    digits=4,
    zero_division=0
))
print("\n✅ 딥러닝(RoBERTa) 모델 평가 결과가 'report_dl' 변수에 저장되었습니다.")


# ============================================================================
# 10. [🌟 신규] 모델 성능 비교 및 시각화 (★ ★ ★ 버그 수정 ★ ★ ★)
# ============================================================================
print("\n" + "="*80)
print("📊 10. [종합] 모델 성능 비교 및 시각화")
print("="*80)

# (★ 메모리 최적화) 6번에서 삭제된 변수들 재사용을 위해 재생성
print("... 베이스라인(LR) 모델 예측값 재생성 (시각화용) ...")
X_test_text_viz = test_ds_processed['text']
y_true_baseline_viz = test_ds_processed['label']
X_test_tfidf_viz = tfidf_vectorizer.transform(X_test_text_viz)
y_pred_baseline_viz = lr_clf.predict(X_test_tfidf_viz)
del X_test_text_viz, X_test_tfidf_viz # 사용 후 즉시 삭제
gc.collect()
print("✅ 베이스라인 예측값 준비 완료")


# 10-1. 종합 성능 비교표 (DataFrame)
print("\n### 1. 종합 성능 비교표 (Weighted Avg.)\n")
metrics_lr = report_lr['weighted avg']
metrics_dl = report_dl['weighted avg']
comparison_df = pd.DataFrame({
    'Baseline (TF-IDF + LR)': metrics_lr,
    'KLUE-RoBERTa': metrics_dl
}).T[['precision', 'recall', 'f1-score', 'support']]
print(comparison_df.to_string(float_format="{:.4f}".format))


# 10-2. 종합 성능 시각화 (Bar Chart)
print("\n\n### 2. 종합 성능 시각화 (F1 / Precision / Recall)\n")
df_long = comparison_df.drop(columns='support').reset_index().melt(
    'index', var_name='Metric', value_name='Score'
)
df_long.rename(columns={'index': 'Model'}, inplace=True)
plt.figure(figsize=(10, 6))
sns.barplot(data=df_long, x='Metric', y='Score', hue='Model', palette='muted')
plt.title('Baseline vs. KLUE-RoBERTa (Weighted Avg. Metrics)', fontsize=16, fontweight='bold')
plt.ylabel('Score')
plt.xlabel('Metric')
plt.legend(title='Model', loc='lower right')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()


# 10-3. Side-by-Side 혼동 행렬 (Confusion Matrix) 비교
print("\n\n### 3. 모델별 혼동 행렬 비교 (Normalized)\n")

# (★ ★ ★ 버그 수정 ★ ★ ★)
# 'ticklabels='를 'xticklabels='로 수정합니다.
def plot_confusion_matrix_custom(ax, y_true, y_pred, classes, title):
    cm = confusion_matrix(y_true, y_pred, normalize='true')
    sns.heatmap(cm, annot=True, fmt='.2f', cmap='Blues', square=True,
                xticklabels=classes, # ★ 수정됨
                yticklabels=classes,
                cbar=False, ax=ax)
    ax.set_ylabel('Actual Label', fontsize=12, fontweight='bold')
    ax.set_xlabel('Predicted Label', fontsize=12, fontweight='bold')
    ax.set_title(title, fontsize=14, pad=20)
    ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')
    ax.set_yticklabels(ax.get_yticklabels(), rotation=0)

fig, axes = plt.subplots(1, 2, figsize=(16, 7))

# 1번 플롯: 베이스라인 (LR)
plot_confusion_matrix_custom(
    axes[0],
    y_true_baseline_viz,
    y_pred_baseline_viz,
    english_labels_ordered,
    'Baseline (TF-IDF + LR)'
)

# 2번 플롯: 딥러닝 (RoBERTa)
plot_confusion_matrix_custom(
    axes[1],
    y_true_dl,
    y_pred_dl,
    english_labels_ordered,
    'KLUE-RoBERTa'
)

fig.suptitle('Model Comparison: Confusion Matrix (Normalized)', fontsize=18, fontweight='bold', y=1.05)
plt.tight_layout()
plt.show()

del y_true_baseline_viz, y_pred_baseline_viz, y_true_dl, y_pred_dl
gc.collect()


# ============================================================================
# 11. 최종 모델 저장
# ============================================================================
print("\n" + "="*80)
print("🏆 11. 최종 모델 저장")
print("="*80)

trainer.save_model(str(SAVE_BASE_PATH))
tokenizer.save_pretrained(str(SAVE_BASE_PATH))
with open(SAVE_BASE_PATH / "label_encoder.pkl", 'wb') as f:
    pickle.dump(label_encoder, f)

print(f"💾 모델 저장 완료: {SAVE_BASE_PATH}")

# 불필요한 체크포인트 폴더 삭제
shutil.rmtree("/content/news-classifier-checkpoints", ignore_errors=True)
print("✅ 로컬 체크포인트 삭제 완료")
print("🎉 모든 작업 완료!")

Writing /content/news-classifier/notebooks/01_train_full_pipeline.ipynb
